# Overview

About Tensors see [What are tensors?](https://www.kaggle.com/code/aisuko/what-are-tensors)

We are going to use the FashionMNIST dataset to train a new simple model and optimize it using PyTorch.

Pytorch has two primitives to work with data:

* `torch.utils.data.DataLoader`
* `torch.utils.data.Daset.Dataset`

They stores the samples and their corresponding labels, and `DataLoader` wraps an iterable around the `Dataset`.

## TORCH.UTILS.DATA

`torch.utils.data.DataLoader` class is the the heart of PyTorch data loading utility. It supports for:

* map-style and iterable-style datasets
* customizing data loading order
* automatic batching
* single-and multi-process data loading
* automatic memory pining

More detail in the notebook [DataLoader in PyTorch](https://pytorch.org/docs/stable/data.html).

PyTorch offers domain-specific libraries such as:
* TorchText
* TorchVision
* TorchAudio

All of which include datasets. We wil be using a TorchVision dataset. The list of `torchvision.datasets` module contains in [here](https://pytorch.org/vision/stable/datasets.html).

# Download the dataset

Every TorchVision `Dataset` includes two arguments:
* `transform`
* `target_transform`

to modify the samples and lables respectively.

In [ ]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data=datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data=datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

# Loading the dataset

We pass the Dataset as an argument to DataLoader. This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, each element in the dataloader iterable will return a batch of 64 features and labels.

In [ ]:
from torch.utils.data import DataLoader

batch_size=64

train_dataloader=DataLoader(training_data, batch_size=batch_size)
test_dataloader=DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N,C,H,W]:{X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")

# Define a Model

To define a neural network in PyTorch, we create a class that inherits from [nn.Module](). We define the layers of the network in the `__init__` function and specify how data will pass through the network in the `forward` function. To accelerate operations in the neural network, we move it to the GPU or MPS if avaliable.

In [ ]:
device=("cuda" if torch.cuda.is_avaliable() else "cpu")
device

In [ ]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten=nn.Flatten()
        self.linear_relu_stack=nn.Sequantial(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    
    def forward(self, x):
        x=self.flatten(x)
        logits=self.linear_relu_stack(x)
        return logits
model=NeuralNetwork().to(device)
model

# Optimizing the Model Parameters

To train a model, we need a [loss function]() and [optimizer]().

In [ ]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(), lr=1e-3)

# Training the model

In a single training loop, the model maskes predictions on the training dataset(fed to it in batches), and backpropagates the prediction error to adjust the model's parameters

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size=len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X,y =X.to(device), y.to(device)
        
        # compute prediction error
        pred=model(X)
        loss=loss_fn(pred,y)

        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch%100==0:
            loss.current=loss.item(), (batch+1) *len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")
            

# Evaluate the model

In [ ]:
def test(dataloader, model, loss_fn):
    size=len(dataloader.dataset)
    num_batches=len(dataloader)
    model.eval()
    test_loss, correct=0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y =X.to(device), y.to(device)
            pred=model(X)
            test_loss+=loss_fn(pred, y).item()
            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss/=num_batches
    correct/=size
    print(f"Test Error:\n Accuracy: {(100*correct):>0.1f}%, Avg loss:{test_loss:>8f} \n")

The training process is conducted over several iterations(epochs). During each epoch, the model learns parameters to make better predictions. We print the model's accuracy and loss at each spoch; we'd like to see the accuracy increase and the loss decrease with every epoch.

In [ ]:
epochs=5

for t in range(epochs):
    print(f"Epoch {t+1}\n------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

In [ ]:
torch.save(model.state_dict,"simple_model.pth")

In [ ]:
!ls

# Loadinf models

The process of loading a model includes re-creating the model structure and loading the state dictionary into it.

In [ ]:
import gc

def model
gc.collect()
torch.cuda.empty_cache()

In [ ]:
model=NeuralNetwork().to(device)
model.load_state_dict(torch.load("simple_model.pth"))